In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!apt install mongodb >log
!service mongodb start
!wget http://alazar.people.ysu.edu/msr14data/datasets/eclipse.tar.gz
!tar zxvf eclipse.tar.gz
!mongorestore
!pip install --upgrade gensim



 * Starting database mongodb
   ...done.
--2022-01-26 22:28:33--  http://alazar.people.ysu.edu/msr14data/datasets/eclipse.tar.gz
Resolving alazar.people.ysu.edu (alazar.people.ysu.edu)... 150.134.10.16
Connecting to alazar.people.ysu.edu (alazar.people.ysu.edu)|150.134.10.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 266976961 (255M) [application/x-gzip]
Saving to: ‘eclipse.tar.gz’

eclipse.tar.gz      100%[===================>] 254.61M  3.63MB/s    in 52s     

2022-01-26 22:29:26 (4.90 MB/s) - ‘eclipse.tar.gz’ saved [266976961/266976961]

dump/eclipse/pairs.metadata.json
dump/eclipse/
dump/eclipse/system.indexes.bson
dump/eclipse/clear.bson
dump/
dump/eclipse/initial.bson
dump/eclipse/clear.metadata.json
dump/eclipse/initial.metadata.json
dump/eclipse/pairs.bson
2022-01-26T22:29:36.490+0000	using default 'dump' directory
2022-01-26T22:29:36.490+0000	preparing collections to restore from
2022-01-26T22:29:36.491+0000	reading metadata for eclipse.initial 

In [ ]:
from pymongo import MongoClient
import gensim
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from pymongo.errors import DuplicateKeyError

import warnings
warnings.filterwarnings('ignore')


try:
    nltk.data.find('./corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('./corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

client = MongoClient()
db = client['eclipse']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
'''from pymongo import MongoClient
from processing import processDocument
from tqdm import tqdm
from google.colab import files


client = MongoClient()
db = client['eclipse']
collection = db['initial']

fields = {'_id': 0,'bug_id':1,'description':1,'short_desc':1}
BATCH_SIZE = 50000

collection.create_index('bug_id')

with open('sentencesB1_50000.txt','w') as file:
    cursor = collection.find({},fields,no_cursor_timeout=True)
    cursor.skip(50000)
    doc = None
    string = ""
    count = 1
    while count<=1:
        string = ""
        for _ in tqdm(range(BATCH_SIZE)):
            doc = next(cursor,None)
            #print("{}\r".format(doc['bug_id']),end='')
            if doc:
                string += str({doc["bug_id"]:list(processDocument(doc,2))}) + '\n'
            else:
                break
        file.write(string)
        print("Batch {} done!".format(count))
        count += 1
        if doc == None:
            break
    cursor.close()
file.close
client.close()
files.download('sentencesB1_50000.txt');'''

In [ ]:
# Load pre-trained Word2Vec model.
model = gensim.models.Word2Vec.load("/gdrive/MyDrive/trained_skip_gram/trained_sg_270122_2_corpus_ds.model", mmap = 'r')

In [ ]:
cur = db.pairs.find()
collection = db['initial']

In [ ]:
# To generate a single vector for a document by combining vectors of the words of the document
def mean_vector(model,wordsTokens:list):
    if len(wordsTokens) > 0:
        return np.mean(model.wv[wordsTokens],axis = 0)
def processDocument(document):
    
    contentofInterest = document['description'] + document['short_desc']  # COI
    
    cleanedCOI = cleaning(contentofInterest)
    tokenized_COI = tokenize(cleanedCOI)
    COI_without_sw = removeStopwords(tokenized_COI)
    
    return COI_without_sw
def cleaning(document):
    try:
        # To remove timestamp and date
        document = re.sub(r"(([A-Z]{2,}\s)*(\([0-9\/]+(\s)*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2} (AM|PM)*)\))",'',document)
        
        # To remove XML code
        document = re.sub(r"(\<\?xml[a-zA-Z0-9\.\s\=\?\"\-\>\\n\<\:\/\_]*(\<\/)[a-zA-Z0-9\:\_]*\>)",'',document)
        
        
        # To remove any hyperlinks
        document = re.sub(r"(http|https):(\/{2})(www\.)([a-zA-z0-9]*\.([a-z]*)(\.)*)",'',document)
        document = re.sub(r"(http|https):(\/{2}[a-zA-Z0-9\.\-\/\:\?\=]*)",'',document)
        
        
        # To remove error string like in bug_id:99873
        # (e.g-line: 62\n\tServerTypeDefinitionUtil.getServerClassPathEntry)
        document = re.sub(r"(line\:\s[0-9]*([\\n\\t])*([a-zA-Z0-9\(\)\$\[\]\\n\\t\s]*\.[a-zA-Z0-9\(\)\,\\n\[\]\s\$\_]*\))*)",
                       '',document)
        
        # To remove the org. from error eg(org.eclipse.ui.internal.Workbench.createAndRunWorkbench(Workbench.java:366))
        document = re.sub(r"((\()*(org|sun|java|junit|e.g)\.[a-zA-Z0-9\.\$\(\:\s\-\,\_\\\<\>]*(\)+|))",'',document)
        
        # To remove string like 'Log: Wed Jun 06 12:51:50 EDT 2001'
        document = re.sub(r"(Log\:[a-zA-Z0-9\s\:]+([0-9]{4}))",'',document)
        
        # To remove strings like /usr/lib/libthread.so.1  (bug_id:33431)
        document = re.sub(r"((\/opt|\/usr)\/[a-zA-Z0-9\/\.\_\,\-]*)",'',document)
        
        # To remove hexadecimal numbers (bug_id:33431)
        document = re.sub(r"(0[xX][a-fA-F0-9]+)",'',document)
        
        # To remove cpp,c,java code
        document = re.sub(r"((\()*[a-zA-Z]+\.(cpp|java)[a-zA-Z0-9\:\\n\#\s\<\>\;\(\,\*\)\{\"\/\+\.\-\_\\n\=]*(\})*)",
                       '',document)
        
        # To remove text between {}
        document = re.sub(r"(\{[a-zA-Z0-9\s\(\)\\n\\t\{\:\<\-\>\=\'\[\]\"\|\*\.\;\,\?]+\})",'',document)
        
        # To remove testcase(check bug report no:99844)
        document = re.sub(r"(Testcase\:.*\})",'',document)
        
        # To remove all text within () or [] or <>
        document = re.sub(r"(\([a-zA-Z0-9\s\+\*\.\,\<\-\>\?\\n\-\'\_\/\$\[\]\(\"\:\#\;\%\!]*\)+|(\[[a-zA-Z0-9\s\:]*\])|(\<[a-zA-Z0-9\_\.\s\:\<\,]*\>+))",
                       '',document)
        
        # To remove alphanumeric string like 1GE8YMJ:
        document = re.sub(r"([0-9][a-zA-Z0-9]{6}(\:)*)",'',document)
        
        # To remove string starting with CVS/
        document = re.sub(r"(CVS(\/)*([a-zA-Z]{1,15}\.[a-zA-Z]*)*)",'',document)
        
        # To remove file name 'org.eclipse.gmt.am3.usecase.osgipluginmanagement.zip'
        document = re.sub(r"(org\.[a-zA-Z0-9\.\$\=\_\(\:\s]*(zip|gz|tar))",'',document)
        
        # To remove strings like 'Authors: Mathieu Vénisse & Guillaume Doux'
        document = re.sub(r"(Authors\:[\sA-Za-z\u00C0-\u00ff\&]+)",'',document)
        
        # To remove string 'Best regards' and NOTES:
        document = re.sub(r"((Best regards\,.+\.)|(NOTES\:))",'',document)
        
        # To remove strings like OS=linux, ARCH=x86
        document = re.sub(r"([A-Z\.]*(\=)+[a-zA-Z0-9\s\.\_]*)",'',document)
        
        # To remove strings like (- v, - y)
        document = re.sub(r"((\-)+(\s)*[a-zA-Z]\W)",' ',document)
        
        # To remove string like ITPVCM:WINNT
        document = re.sub(r"([a-zA-Z]+:[a-zA-Z]+)",'',document)
        
        # To change words like Don't to Dont
        document = re.sub(r"(\')",'',document)
        
        # To remove email
        document = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b",'',document)
        
        # To remove windows path
        document = re.sub(r"[a-zA-Z]:[\\]{2}(?:[a-zA-Z0-9\.]+[\\\/]{1,2})*",'',document)
        
        # To remove \\
        document = re.sub(r"(\\){2}",'',document)
        
        # To remove other special chacters
#         document = re.sub(r"[\-\'\:\?\/\[\]\"\$\>\<\,\!\+\#\*\_\|\;\}\{\(\)]",' ',document) -> replaced with a short regex
        document = re.sub(r"[^a-zA-Z\s\\]",' ',document)
        
        # To remove string like (STACK 0) (bug_id: 88623)
        document = re.sub(r"([A-Z]+\s[0-9]+)",'',document)
        
        # To remove repetitive characters (bug_id: 170,146)
        document = re.sub(r"\b[\\](\w+)(?:\w+\1\b)+",'',document)
        
        # To remove all spaces greater than 2
        document = re.sub(r"((\s){2,})",' ',document)
        
        
    except TypeError:
        print('TypeError for bug id: ',document['bug_id'])
    except DuplicateKeyError:
        print('DuplicateKeyError for bug id: ',document['bug_id'])
    
    return document
    
def tokenize(document):
    tokens = wordpunct_tokenize(document)
    
    return tokens

def removeStopwords(tokens):
    tokens_without_sw = [word.lower() for word in tokens if not word in stopwords.words()]
    
    return tokens_without_sw

def wordStemming(tokens):
    snow_stemmer = SnowballStemmer(language='english')
    stem_words = [snow_stemmer.stem(word) for word in tokens]
    
    return stem_words


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71271'),
 'bug1': 214301,
 'bug2': 214611,
 'dec': 1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214301'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214611'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.918087]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71272'),
 'bug1': 214623,
 'bug2': 214825,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214623'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214825'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.80060947]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71273'),
 'bug1': 214445,
 'bug2': 214451,
 'dec': 1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214445'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214451'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.9572139]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71275'),
 'bug1': 214181,
 'bug2': 214620,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214181'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214620'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.69625145]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71276'),
 'bug1': 214577,
 'bug2': 214179,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214577'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214179'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.7372564]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71277'),
 'bug1': 214862,
 'bug2': 214759,
 'dec': 1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214862'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214759'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.9102196]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71278'),
 'bug1': 215000,
 'bug2': 214738,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'215000'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214738'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.7884146]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71279'),
 'bug1': 214510,
 'bug2': 214318,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214510'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214318'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.6877037]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f7127a'),
 'bug1': 214971,
 'bug2': 214241,
 'dec': -1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214971'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214241'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.6737332]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71281'),
 'bug1': 214305,
 'bug2': 214303,
 'dec': 1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214305'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214303'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.9686242]]


In [ ]:
cur.next()

{'_id': ObjectId('5307f231166663d2e3f71285'),
 'bug1': 214306,
 'bug2': 214305,
 'dec': 1}

In [ ]:
bug_doc1 = collection.find_one({'bug_id':'214306'})
bug1 = processDocument(bug_doc1)
bug_doc2 = collection.find_one({'bug_id':'214305'})
bug2 = processDocument(bug_doc2)
vector1 = mean_vector(model,bug1)
vector2 = mean_vector(model,bug2)
print(cosine_similarity([vector1],[vector2]))

[[0.98087925]]
